In [5]:
import numpy as np
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation, Dropout, Convolution2D, GlobalAveragePooling2D
from keras.models import Sequential
import tensorflow as tf
import tensorflow.keras.applications.mobilenet
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Input


In [6]:

dataset_path = '/Users/sivaprasanth/Downloads/Hackathon/Skin_sample_Data/train'

classes = {'Actinic keratosis': 0,'Atopic Dermatitis': 1,'Benign keratosis': 2,'Dermatofibroma': 3,'Melanocytic nevus': 4,'Melanoma':5,'Squamous cell carcinoma': 6,'Tinea Ringworm Candidiasis':7,
          'Vascular lesion': 8
}


In [7]:

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=20,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory('/Users/sivaprasanth/Downloads/Hackathon/Skin_sample_Data/train',
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 shuffle = True,
                                                 class_mode = 'categorical')


Found 697 images belonging to 9 classes.


In [8]:

from sklearn.utils import shuffle

import cv2
img_size=224
def resize_to_square(image, size):
    h, w, c = image.shape
    ratio = size / max(h, w)
    resized_image = cv2.resize(image, (int(w*ratio), int(h*ratio)), cv2.INTER_AREA)
    return resized_image
def padding(image, min_height, min_width):
    h, w, c = image.shape

    if h < min_height:
        h_pad_top = int((min_height - h) / 2.0)
        h_pad_bottom = min_height - h - h_pad_top
    else:
        h_pad_top = 0
        h_pad_bottom = 0

    if w < min_width:
        w_pad_left = int((min_width - w) / 2.0)
        w_pad_right = min_width - w - w_pad_left
    else:
        w_pad_left = 0
        w_pad_right = 0

    return cv2.copyMakeBorder(image, h_pad_top, h_pad_bottom, w_pad_left, w_pad_right, cv2.BORDER_CONSTANT, value=(0,0,0))

def load_data(directory):
    output = []
    images = []
    labels = []
    file_names = []
    for folder in os.listdir(directory):
        print("***",folder)
        curr_label=classes[folder]
        for file in os.listdir(directory + "/" + folder):
            img_path = directory + "/" + folder + "/" + file
            curr_img = cv2.imread(img_path)
            curr_img = resize_to_square(curr_img, img_size)
            curr_img = padding(curr_img, img_size, img_size)
            images.append(curr_img)
            labels.append(curr_label)
            file_names.append(file)
    images, labels, file_names = shuffle(images, labels, file_names, random_state=817328462)     ### Shuffle the data !!!
    images = np.array(images, dtype = 'float32') ### Our images
    labels = np.array(labels, dtype = 'int32')   ### From 0 to num_classes-1!

    return images, labels, file_names

print(dataset_path)
images, labels, file_names=load_data(dataset_path)


/Users/sivaprasanth/Downloads/Hackathon/Skin_sample_Data/train
*** Melanocytic nevus
*** .DS_Store


KeyError: '.DS_Store'

In [ ]:

x_data = images
y_data = labels
print(x_data.shape)
print(y_data.shape)
print(y_data)




(697, 224, 224, 3)
(697,)
[3 4 1 1 7 5 3 8 0 0 4 0 6 6 4 6 6 5 2 6 4 2 0 7 0 4 8 6 8 6 8 6 6 6 0 3 3
 8 7 4 3 0 7 4 7 0 4 1 2 6 1 8 7 4 1 5 6 2 4 7 6 5 3 7 6 8 0 5 7 3 4 4 8 5
 8 5 7 8 0 0 6 8 4 7 2 7 3 3 2 0 3 5 6 6 3 5 2 0 1 4 8 3 3 2 2 4 2 4 5 0 4
 2 6 0 3 1 8 7 6 6 5 8 8 8 7 8 0 8 4 4 1 4 0 0 4 0 6 0 3 1 3 6 2 6 6 3 8 3
 5 8 6 2 8 8 7 0 3 4 3 7 0 2 8 1 5 2 4 4 6 2 8 1 5 1 4 8 3 2 6 0 3 1 7 0 3
 5 4 2 5 7 0 5 0 5 7 5 8 7 5 1 4 0 7 7 3 1 0 2 1 4 3 3 4 2 1 8 5 3 2 8 4 4
 2 6 6 8 5 2 4 5 6 6 2 4 1 1 4 6 1 1 8 8 4 7 4 8 0 6 0 7 5 4 2 8 6 0 8 8 2
 5 7 2 0 6 5 7 6 8 5 4 1 4 7 2 0 0 4 0 6 6 1 5 2 6 3 1 3 6 1 5 6 7 8 7 4 4
 3 1 5 1 8 4 2 4 2 4 5 5 5 6 4 0 0 1 3 6 5 6 0 5 1 1 2 8 2 3 3 6 1 7 8 7 1
 3 1 8 6 8 2 1 5 6 5 3 8 2 6 4 5 3 2 8 3 2 0 5 8 0 7 2 2 6 1 5 6 1 5 4 6 6
 1 8 7 2 1 5 5 5 6 8 2 0 3 5 0 1 5 1 2 6 1 8 0 1 2 0 2 6 8 5 5 0 8 5 8 3 2
 2 3 0 2 3 6 0 1 3 1 5 2 4 3 1 6 4 0 7 1 5 4 4 3 8 3 4 3 0 3 1 1 7 3 7 1 4
 4 3 5 1 6 8 2 8 5 1 8 8 0 5 6 4 2 7 5 1 1 4 3 3 7 2 0 1 3 7 2 6 1 5 0 6 6

In [ ]:


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.layers import Conv2D,MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.optimizers import SGD
from keras.utils import to_categorical


In [ ]:

num_classes = 9
Y = to_categorical(y_data,num_classes)
X = x_data


In [ ]:


from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)



(522, 224, 224, 3)
(522, 9)
(175, 224, 224, 3)
(175, 9)


In [ ]:

TRAINING_NUMBER = len(y_train)
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.005
MOMENTUM = 0.9
DECAY = 1e-6
num_channel = 3


In [ ]:

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import load_model
#from tensorflow.keras.models import loadLayersModel

#VGG16 = await tf.loadLayersModel('/Users/sivaprasanth/Desktop/Hackathon/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

image_input = Input(shape=(img_size,img_size,num_channel))
vgg_mod = VGG16(input_tensor=image_input, include_top=False,weights='imagenet')
vgg_mod.summary()


Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5: None -- [Errno 8] nodename nor servname provided, or not known

In [ ]:

import tensorflow as tf
add_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=vgg_mod.output_shape[1:]), # the nn will learn the good filter to use
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(9, activation=tf.nn.softmax)
])

model_vgg16 = Model(inputs=vgg_mod.input, outputs=add_model(vgg_mod.output))
model_vgg16.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'],)
model_vgg16.summary()


In [ ]:

from keras.callbacks import ReduceLROnPlateau

callbacks_list = [ReduceLROnPlateau(monitor='loss',factor=0.2,patience=3)]

hist_1=model_vgg16.fit(X_train,y_train,
                       batch_size=BATCH_SIZE,
                       epochs=EPOCHS,
                       callbacks = callbacks_list,
                       verbose=1)

#Testing result
score = model_vgg16.evaluate(X_test, y_test, verbose=0)


In [ ]:

print('Test loss:', score[0])
print('Test accuracy:', score[1])